In [22]:
import numpy as np
import math as mh
import orix
import matplotlib.pyplot as plt

from operator import itemgetter
from matplotlib import colormaps

#from orix import plot, sampling
from orix.crystal_map import Phase
from orix.quaternion import Orientation, Rotation, symmetry, Quaternion
from orix.vector import Miller, Vector3d
from diffpy.structure import Atom, Lattice, Structure
from orix.quaternion.symmetry import Oh

from orix.io import plugins

In [23]:

def asCartesian(rthetaphi):
    #takes list rthetaphi (single coord)
    r       = rthetaphi[0]
    theta   = rthetaphi[1]* np.pi/180 # to radian
    phi     = rthetaphi[2]* np.pi/180
    x = r * mh.sin( theta ) * mh.cos( phi )
    y = r * mh.sin( theta ) * mh.sin( phi )
    z = r * mh.cos( theta )
    return [x,y,z]

def asSpherical(xyz):
    #takes list xyz (single coord)
    x       = xyz[0]
    y       = xyz[1]
    z       = xyz[2]
    r       =  mh.sqrt(x*x + y*y + z*z)
    theta   =  mh.acos(z/r)*180/ np.pi #to degrees
    phi     =  mh.atan2(y,x)*180/ np.pi
    return [r,theta,phi]

In [24]:
#Paramètres figures

plt.rcParams.update(
    {
        "figure.figsize": (10, 10),
        "figure.facecolor":"w",
        "font.size": 10,
        "axes.grid": True,
        "lines.markersize": 10,
        "lines.linewidth": 2,
    }
)


In [ ]:
#test uvw direction normal to plane hkl 
#maille hexagonale P63mc
maille=Lattice(3.25,3.25,5.2,90,90,120)
laue_group=symmetry.C3h
ZnO_hex=Phase(structure=Structure(lattice=maille),point_group=laue_group)

Milhkl=Miller(hkl=[1,0,0],phase=ZnO_hex)
print("Le plan hkl est",Milhkl.hkl)
Miluvw=Miller(uvw=Milhkl.uvw,phase=ZnO_hex)
print("La normal uvw correspondante est",Miluvw.round(10).uvw)

#Inversement

Miluvw=Miller(uvw=[2,1,0],phase=ZnO_hex)
print("La direction uvw est",Miluvw.uvw)
Milhkl=Miller(hkl=Miluvw.hkl,phase=ZnO_hex)
print("Ce qui est la normal aux plans",Milhkl.round(10).hkl)




Le plan hkl est [[ 1.00000000e+00 -2.77555756e-17  0.00000000e+00]]
La normal uvw correspondante est [[2. 1. 0.]]
La direction uvw est [[2. 1. 0.]]
Ce qui est la normal aux plans [[ 1.00000000e+00 -2.77555756e-17  0.00000000e+00]]


In [ ]:
#Elastic constant Zno
c11=209.7
c12=121.1
c13=105.1
c22=c11
c33=210.9
c44=42.47
c66=44.29

#Elastic matrix
C=np.array([[c11,c12,c13,0,0,0],[c12,c11,c13,0,0,0],[c13,c13,c33,0,0,0],[0,0,0,c44,0,0],[0,0,0,0,c44,0],[0,0,0,0,0,c66]])
S=np.linalg.inv(C)

s11=S[0][0]
s12=S[0][1]
s13=S[0][2]
s14=S[0][3]
s33=S[2][2]
s44=S[3][3]

#E along direction
#Plans
planhkil=[1,0,-1,0]
print("Le plan hkli est",planhkil)

#CalculE
Milhkil=Miller(hkil=planhkil,phase=ZnO_hex)
print("Le plan hkl correspondant est",Milhkil.hkl)
Miluvw=Miller(uvw=Milhkl.uvw,phase=ZnO_hex)
Miluvtw=Miller(UVTW=Milhkl.UVTW,phase=ZnO_hex)
print("La normal uvw correspondante est",Miluvw.round(10).uvw, "ou en 4 indices", Miluvtw.round(10).UVTW)

Mydirect=Miluvw.uvw[0]
NormMydirect=np.linalg.norm(Mydirect)
Mydirect=Mydirect/NormMydirect
l1=Mydirect[0]
l2=Mydirect[1]
l3=Mydirect[2]

invE=(1-l3**2)**2*s11 + l3**4*s33 + l3**2*(1-l3**2)*(2*s13+s44) + 2*l2*l3*(3*l1**2-l2**2)*s14
E=1/invE
print("Le module dans la direction",Miluvw.round(10).uvw, "donc normale aux plans", Milhkil.round(10).hkl, "est",round(E,1), "GPa")



Le plan hkli est [0, 0, 0, 1]
Le plan hkl correspondant est [[0. 0. 1.]]
La normal uvw correspondante est [[2. 1. 0.]] ou en 4 indices [[ 1.  0. -1.  0.]]
Le module dans la direction [[2. 1. 0.]] donc normale aux plans [[0. 0. 1.]] est 127.3 GPa


In [27]:
from mechelastic.core import ELATE
from mechelastic.parsers import QE_thermo_pw_Parser
import pyvista as pv


ModuleNotFoundError: No module named 'mechelastic'

In [ ]:

#Elastic constant Zno
c11=209.7
c12=121.1
c13=105.1
c22=c11
c33=210.9
c44=42.47
c66=44.29

#Elastic matrix

C=np.array([[c11,c12,c13,0,0,0],[c12,c11,c13,0,0,0],[c13,c13,c33,0,0,0],[0,0,0,c44,0,0],[0,0,0,0,c44,0],[0,0,0,0,0,c66]])
S=np.linalg.inv(C)

elastic_tensor =[[c11,c12,c13,0,0,0],[c12,c11,c13,0,0,0],[c13,c13,c33,0,0,0],[0,0,0,c44,0,0],[0,0,0,0,c44,0],[0,0,0,0,0,c66]]

density = None # in kg/m^3

elate = ELATE(s = elastic_tensor, density = density)


elate.print_properties()



In [ ]:
#available with “BULK”, POISSON” , “SHEAR”, “YOUNG” , “LC” ,”PUGH_RATIO”
fig = elate.plot_2D(elastic_calc = "YOUNG", show = True )

In [ ]:
#available with “BULK”, POISSON” , “SHEAR”, “YOUNG” , “LC” ,”PUGH_RATIO”
meshes = elate.plot_3D(elastic_calc="YOUNG",normal=[1,1,0],npoints=500, show = True)

